In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time
import json
import glob
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from nutil import normImage
import torch.utils.data as data_utils
from dep.unet import ResNetUNetNew as ResNetUNet
from nutil import show_full_render1
from augment import generateImages
from nutil import get_emb_vis, huber, sample_images_at_mc_locs, returnCrossEntropy,returnCrossEntropyWithNeg
from nerf import NeuralRadianceFieldFeat
from pren import ImplicitRendererStratified, EmissionAbsorptionRaymarcherStratified
from PIL import Image
from IPython import display
from tqdm.notebook import tqdm
from cowrendersynth import generate_cow_rendersWithRT, generate_bop_realsamples
from torch.utils.tensorboard import SummaryWriter
from nutil import mip360loss
from pytorch3d.structures import Volumes
from pytorch3d.transforms import so3_exp_map

from sklearn.neighbors import KDTree
from dataGen import AugmentedSamples
import pytorch3d,cv2
from nutil import rotfromeulernp
import argparse
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    PerspectiveCameras,
    NDCMultinomialRaysampler,
    MonteCarloRaysampler,
    EmissionAbsorptionRaymarcher,
    ImplicitRenderer,
    RayBundle,
    ray_bundle_to_ray_points,
)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
objid="1"
expID = "ruapc/"+objid+"poseEst"
batch_size = 16
sampleSize = 1024
key_noise = 1e-3
Siren=True
datasetPath = "bop/ruapc/"
imD = 224
in_ndc = False


In [4]:
torch.manual_seed(1)
# Instantiate the radiance field model.
neural_radiance_field = NeuralRadianceFieldFeat(siren=Siren)
encoder_rgb = ResNetUNet(n_class=(13),n_decoders=1,)  ##?n_class =13 - dm cos here is the query image with embedding size =13 ddo
#encoder_rgb2 = ResNetUNet(n_class=(3),n_decoders=1,)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/damvan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [5]:
# load nerf model as key model
data_nerf = torch.load(expID+"/nerflatest.pth")	
neural_radiance_field.load_state_dict(data_nerf["model_state_dict"])
# load CNN as query model	
data_rgb = torch.load(expID+"/encoderRGBlatest.pth")	
encoder_rgb.load_state_dict(data_rgb["model_state_dict"])	
neural_radiance_field=neural_radiance_field
encoder_rgb=encoder_rgb

In [6]:
#set models to eval
neural_radiance_field.eval()
encoder_rgb.eval()
print("continue")

continue


In [7]:
# load images from second sequence
fsamps = int(2561 * 0.5)
fewIds = np.arange(int(2561 * 0.5)) +1280
target_images, target_silhouettes, RObj, TObj, KObj, fewIds = generate_bop_realsamples(datasetPath,objid=objid, crop=True,maskStr="mask",offset=5, synth=False,makeNDC=in_ndc,dataset="tless", maxB=imD, background=False, fewSamps=True, fewCT=fsamps, fewids=fewIds)

In [8]:
## load fullNegVec
if os.path.exists(expID+"/negVec.npy"):
    fullNegVec= torch.from_numpy(np.load(expID+"/negVec.npy").astype("float32")).unsqueeze(0)
negSamps = torch.randperm(fullNegVec.shape[1], dtype=torch.int64)[0:batch_size*sampleSize]
negVec=fullNegVec[:,negSamps].clone()
# negVec+= torch.randn_like(negVec) * key_noise # negvec =negvec +negvec*keynoise
print(fullNegVec.shape)
print(negVec.shape) #only take 16384 points from the all the 3D points- to reduce computation at the end


torch.Size([1, 79972, 3])
torch.Size([1, 16384, 3])


In [9]:
target_images.shape[0]

1280

In [10]:
input =torch.movedim(target_images, 3, 1)
input.shape[0]

1280

In [11]:
with torch.no_grad():
    queries = [encoder_rgb.to(device)(input[i].unsqueeze(0).to(device)) for i in range(input.shape[0])]
    feats = neural_radiance_field.batched_customForward(negVec)


In [12]:
del encoder_rgb

In [13]:
queries = torch.stack([tensor.to('cpu') for tensor in queries]).squeeze()
print(queries.shape)
print(feats.shape)

torch.Size([1280, 13, 224, 224])
torch.Size([1, 16384, 13])


In [14]:
#first image of second sequence as 1280th image 
ith_queries = queries[0].reshape(-1,13)
resized_feats = feats.squeeze() 
print(ith_queries.shape)
print(resized_feats.shape)

torch.Size([50176, 13])
torch.Size([16384, 13])


In [15]:
def getCors(queries, feats, leaves=1):
    similarity_mat=(queries @ feats.T)
    cMat = torch.log_softmax(similarity_mat, dim=-1)
    ids=torch.argmax(cMat, dim=-1)
    #vals, idx = torch.topk(cMat, k=leaves, dim=-1)
    # if leaves == 1:
    #     return idx[..., 0].cpu(), vals
    # else:
    #     return idx.cpu(), vals
    return ids


In [16]:
#obtain id of 3D corresponding in the negVec
ids= getCors(ith_queries, resized_feats)
len(ids)

50176

In [17]:
mask =target_silhouettes[0]
#fisrt convert mask to 0,1
mask[mask < 0.5] = 0
mask[mask >= 0.5] = 1

In [18]:
obj_pixels = [torch.tensor([i, j]) for i in range(224) for j in range(224) if mask[i, j] == 1]  ## pixels inside object
# back_ground = [[i, j] for i in range(224) for j in range(224) if mask[i, j] == 0] ## background
len(obj_pixels)

15554

In [19]:
# get the 3D corres for the obj_pixels
corIds=[ids[pixel[0]*224+pixel[1]] for pixel in obj_pixels]
print(len(corIds))

15554


In [20]:
##finally we have 2D and its 3D Corre
pt2D = torch.stack(obj_pixels, dim = 0).numpy().astype("float32")
pt3D = negVec[:,corIds,:].squeeze().numpy().astype("float32")

In [21]:
# scale back 3D points
diameter = 103.058
pt3D = diameter/0.5*pt3D

In [22]:
## print an example result
print(pt2D[0])
print(pt3D[0])

[11. 99.]
[ -21.538494   44.856907 -178.02055 ]


In [23]:
# rot180 = rotfromeulernp(np.array([0, 0, np.pi]))
# for a in range(target_images.shape[0]):
#   RObj[a] = (RObj[a].T).dot(rot180)
#   TObj[a, 0:2] = -TObj[a, 0:2]
# meshdetails = json.load(open(datasetPath + "/models" + "/models_info.json"))
# diam = meshdetails[objid]['diameter']
# diamScaling=1.8
# offset=0
# scale=diam/diamScaling
# TObj = TObj/scale


# stratified=False
# enable3D=False
# enableSSP=False
# negativeSampling=True
# maskRays=True
# rayFreeze=False

# target_cameras = PerspectiveCameras(device=device, R=torch.from_numpy(RObj.astype("float32")),
#                                     T=torch.from_numpy(TObj.astype("float32")),
#                                     K=torch.from_numpy(KObj.astype("float32")))

In [24]:
def Pose6DEstimator(image_points,world_points):
    # Distortion coefficients (if any)
    dist_coeffs = np.array([0, 0, 0, 0, 0], dtype=np.float32)
    K = np.array([531.15, 0.0, 320.0, 0.0, 531.15, 240.0, 0.0, 0.0, 1.0]).reshape(3,3).astype('float32')
    ransac_reproj_threshold = 5.0  # Threshold for RANSAC reprojection error
    ransac_iterations = 1000  # Number of RANSAC iterations
    # Estimate pose using solvePnPRansac
    success, rvec, tvec, inliers = cv2.solvePnPRansac(world_points, image_points, K, dist_coeffs,
                                                 reprojectionError=ransac_reproj_threshold,
                                                 iterationsCount=ransac_iterations)
    return success, rvec, tvec,inliers

In [25]:
success, rvec, tvec, inliers = Pose6DEstimator(pt2D,pt3D)

In [27]:
R, _ = cv2.Rodrigues(rvec)
t = tvec
R

array([[-0.77541512,  0.63142768, -0.00551972],
       [ 0.63132491,  0.77540491,  0.01326926],
       [ 0.0126586 ,  0.00680445, -0.99989672]])